In [1]:
from funct import * # importing my blackbox

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
'''This section is made for the presientation so we can see what words we are curently looking into'''


from langchain_community.utilities import SQLDatabase 
# this is how we can link the LLM to the SQL DB ^ 

db = SQLDatabase.from_uri("sqlite:///Word_DB.sqlite") # connects here

query = "SELECT word FROM words WHERE fluency = 5;" #getting all word where fluency is set to 5
result = db.run(query)

print("Words with fluency = 5:", result) # prints the selected word

Words with fluency = 5: [(' mais',), (' besoin',), (' seul',), (' fleur',)]


In [3]:
import ollama  # This is where we can get LLMs to store locally

client = ollama.Client() # connection to client

model="gemma3:4b"  # played around alot with a bunch but found that this is the best trade off the power and size so sticking with it

'''This next 3 function i dont know why but can not be moved to funct and need to be in the same file as the pygame when it runs so here are the 
LLM functions we use in AI demo'''


def sentence_gen(word):
    '''creates a simple example sentnece using the word provided as well as words from ones libary based on your fluceny level of them
    - sort by word only above fluency level 3.    or something. 
    '''
    prompt = f"Only return a very basic example of a French sentence that includes this french word: '{word}'. if possible try to use a word from this list that follows but ONLY if it makes sense else do not use any words in list: {result}"

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]


def create_example(word):
    '''Creates a setnence with knowelge of previous messages for the user to decode the message will take in info for words knowen'''

    prompt = f"Generate an easy french sentence that uses this french word: {word}. Make sure to return **ONLY** a sentence in french. If you can feel free to use other word from this list: {result}"

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]  # needs some level of randomization


def test_translation(french, response):

    prompt = f"Is this: {response}. a good english translation of this french sentence {french}? Return *only* 1 sentence saying Correct/A bit off/Incorrect if a but off explain what is missing. if its close enough just say correct."

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]  # needs some level of randomization



# The above are really just varying the prompt and question we send into the LLM as well and the SQL DB of what we are trying to change and play with



In [4]:
# changing the fluency in this part --- will use for the presenetation so we can update the DB and show the varying LLM generation
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Word_DB.sqlite") # connections again
#update_query = "UPDATE words SET fluency = 5 WHERE id = 219;"
#db.run(update_query)

update_all_fluency_query = "UPDATE words SET fluency = 0;" # sets fluecny of all to 0 so we can change to show the varying words
# db.run(update_all_fluency_query)


# the next 3 are fuction of examples of varying words from the SQL DB list -- the words will be showen in the demo as we selction the ex1 2 and 3 showing the changes

def db_example1(): 
    db.run(update_all_fluency_query) # sets all fluency for all words to 0
    list = [177, 220, 219, 59] 

    for i in list: 
        db.run(f"UPDATE words SET fluency = 5 WHERE id = {i}")

    pass


def db_example2():
    db.run(update_all_fluency_query) # sets all fluency for all words to 0
    list = [32, 119, 121, 229]

    for i in list: 
        db.run(f"UPDATE words SET fluency = 5 WHERE id = {i}")

    pass

def db_example3():
    db.run(update_all_fluency_query) # sets all fluency for all words to 0
    list = [216, 223, 217, 226, 228]

    for i in list: 
        db.run(f"UPDATE words SET fluency = 5 WHERE id = {i}")

    pass

db_example2()


In [5]:
# this needs to be in this directly else it won't work on newly added mp3
def mp3_play(word):
    """Same function as in the fuct however needs to be locally so it can be called for some reason?"""
    mp3_path = f"Pronunciations\{word}.mp3"

    if not os.path.exists(mp3_path):  
       print("MP3 file not found.") 
       return
    
    pygame.mixer.init()
    pygame.mixer.music.load(f"Pronunciations\{word}.mp3")
    pygame.mixer.music.play()
    
    print(f"Playing: {word}")

    # this might be required later so i'll keep it open like this
    while pygame.mixer.music.get_busy():  # Keep script running while music plays
        continue
    else:
        pygame.mixer.quit()  # closes it so less of a problem for other functions


    


In [6]:
'''Now This is the part to run after you run all of the above it will open a pygame and our demo'''


import pygame

# starts the pygame through this command
pygame.init()

# Set up the basic display very basic stuff in any pygame tutoral shows
WIDTH, HEIGHT = 500, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pygame Lang-app V0.2")

# Colors delared here 
WHITE = (255, 255, 255)
BACKGROUND = (36, 71, 119)
COLOR_1 = (225, 173, 100)
GRAY = (200, 200, 200)
DARK_GRAY = (150, 150, 150)
LIGHT_GRAY = (200, 200, 200)
BLUE = (0, 122, 204)
BLACK = (0, 0, 0)

# Fonts for easier use later 
font_t = pygame.font.Font(None, 55)
font = pygame.font.Font(None, 40)
button_font = pygame.font.Font(None, 30)

# setting the current screen to main
current_screen = "main"

#for phrase input section --- wrote the sections differnt so not the cleanest shoudl of put in the call for the slides but left them here this is for the phrase part of the code
input_box = pygame.Rect(50, 300, 350, 50)
input_text = ""
active = False
button_checker = pygame.Rect(50, 375, 125, 50)
button_text_checker = "Check"
generation_button_next = pygame.Rect(300, 375, 125, 50)
generation_button_next_text = "Again"
result_text = '--'
current_phrase = "-1"
# All of the above are just varaible i will be using for niche parts of certin screen but i wanted them global so i declared them here

 
# All buttons decleared here
button1_rect = pygame.Rect(100, 350, 300, 50)
button2_rect = pygame.Rect(100, 420, 300, 50)
button3_rect = pygame.Rect(100, 490, 300, 50)
function_button_rect = pygame.Rect(50, 200, 120, 50)
print_button_rect = pygame.Rect(50, 500, 120, 50)
next_w_button_rect = pygame.Rect(300, 500, 120, 50)
p_button_word_react = pygame.Rect(50, 270, 120, 50)
p_button_sent_react = pygame.Rect(300, 270, 120, 50)
back_button_r = pygame.Rect(300, 600, 120, 50)
back_button_p = pygame.Rect(10, 10, 120, 50)

word_index = 0
val = 0                                                             # theses 4 more basic global varibles i will be using
current_phrase = 'base phrase, with some words but not that many'
function_output = ""


def draw_main_screen():
    '''The creation of main main screen function call'''
    screen.fill(BACKGROUND)
    title_text = font_t.render("Cadmus Demo ", True, COLOR_1)   # basic rendering 
    screen.blit(title_text, (110, 100))
    
    pygame.draw.rect(screen, BLUE, button1_rect)
    pygame.draw.rect(screen, BLUE, button2_rect)    # buttons 
    pygame.draw.rect(screen, BLUE, button3_rect)
    
    screen.blit(button_font.render("Study", True, WHITE), button1_rect.move(50, 15))
    screen.blit(button_font.render("DO **NOT** CLICK HERE", True, WHITE), button2_rect.move(20, 15))    # button placements and colors and such
    screen.blit(button_font.render("Complete Phrases", True, WHITE), button3_rect.move(50, 15))


def get_current_word():
    '''Returns the currnet word for focus from list'''
    return words[word_index]



def function_action():
    '''When the function button is clicked this will call the LLM to generate an example setence '''
    global function_output
    function_output = sentence_gen(words[word_index].lower())

def card_display():
    '''Calles the generation def to show the english translation'''
    word = get_current_word()
    global val
    if val == 0:
        return word
    else: 
        return word + " | " + generate_def(word)

def next_word():
    '''Moves the index so we are able to move on to next word'''
    global word_index
    global function_output
    word_index = (word_index + 1) % len(words)
    function_output = ""

def gen_phrase():
    '''generate the examples phrase thats varying slghtly with the current word and is linked to the SQL DB'''
    global word_index
    global current_phrase
    current_phrase = create_example(get_current_word())
    word_index += 1

def wrap_text(text, font, max_width):               # This fuction was found online as i had a problem with displaying the varying text lengths in a box
    """wraps the texts as it varying in length so it always fits within the box provided"""
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + " " + word if current_line else word
        test_surface = font.render(test_line, True, BLACK)
        if test_surface.get_width() <= max_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word 
    
    if current_line: 
        lines.append(current_line)
    
    return lines

def draw_phrases_screen():
    """Now here we generate teh phrase screen"""
    font_n = pygame.font.Font(None, 30)
    screen.fill(BACKGROUND)                         # basic display here
    word_box_rect = pygame.Rect(50, 75, 400, 200)
    pygame.draw.rect(screen, DARK_GRAY, word_box_rect, border_radius=10)

    max_width = word_box_rect.width - 20  # Padding inside the box              # moving of the box here 
    y_offset = word_box_rect.y + 10  # Start inside the box with padding

    lines = wrap_text(current_phrase, font, max_width)  # used for the line wrap fucntion to display varing mength texts

    # Render eahc of the lines
    for line in lines:
        text_surface = font.render(line, True, BLACK)
        screen.blit(text_surface, (word_box_rect.x + 10, y_offset))
        y_offset += font.get_height() 

    # this draws the iunput box
    pygame.draw.rect(screen, LIGHT_GRAY if active else DARK_GRAY, input_box, border_radius=5)
    input_surface = font.render(input_text, True, BLACK)
    screen.blit(input_surface, (input_box.x + 10, input_box.y + 10))

    # does the draw button 
    pygame.draw.rect(screen, BLUE, button_checker, border_radius=5)
    button_surface = font.render(button_text_checker, True, WHITE) 
    screen.blit(button_surface, (button_checker.x + 10, button_checker.y + 10)) 

    #the back button
    pygame.draw.rect(screen, BLUE, back_button_p, border_radius=5)
    button_surface = font.render("Home", True, WHITE) 
    screen.blit(button_surface, (back_button_p.x + 10, back_button_p.y + 10)) 

    # the generate button
    pygame.draw.rect(screen, BLUE, generation_button_next, border_radius=5)
    button_surface = font.render(generation_button_next_text, True, WHITE) 
    screen.blit(button_surface, (generation_button_next.x + 10, generation_button_next.y + 10)) 

    # more text we needed wraped
    lines_2 = wrap_text(result_text, font_n, 400)
    y_offset = 450
    # displaying the multiple lines
    for line in lines_2:
        result_surface = font_n.render(line, True, COLOR_1)
        screen.blit(result_surface, (50, y_offset))
        y_offset += 35  

def draw_review_screen():
    '''the creation of the reveiw screen'''
    screen.fill(BACKGROUND)
    word_box_rect = pygame.Rect(100, 50, 300, 60)          # basic layout
    pygame.draw.rect(screen, DARK_GRAY, word_box_rect, border_radius=10)


    #draws the word in the center of the card /box 
    text_surface = font.render(card_display(), True, BLACK)
    text_rect = text_surface.get_rect(center=word_box_rect.center)
    screen.blit(text_surface, text_rect)

    
    # creation the buttons here
    pygame.draw.rect(screen, BLUE, function_button_rect)
    pygame.draw.rect(screen, BLUE, print_button_rect)
    pygame.draw.rect(screen, BLUE, next_w_button_rect)
    pygame.draw.rect(screen, BLUE, p_button_word_react)
    pygame.draw.rect(screen, BLUE, p_button_sent_react)
    pygame.draw.rect(screen, BLUE, back_button_r)
    
    # rendering here
    screen.blit(button_font.render("Example", True, WHITE), function_button_rect.move(20, 15))
    screen.blit(button_font.render("English", True, WHITE), print_button_rect.move(35, 15))
    screen.blit(button_font.render("Next", True, WHITE), next_w_button_rect.move(35, 15))
    screen.blit(button_font.render("P_Word", True, WHITE), p_button_word_react.move(20, 15))
    screen.blit(button_font.render("P_Sen", True, WHITE), p_button_sent_react.move(20, 15))
    screen.blit(button_font.render("Home", True, WHITE), back_button_r.move(20, 15))

    output_surface = button_font.render((function_output.split(".")[0]).split("?")[0], True, BLACK)
    screen.blit(output_surface, (180, 215)) # cuting and dispalying the text here 

def generate_new_phrase():
    '''Calls gen again shifting the proper global vars'''
    global word_index
    global current_phrase
    current_phrase = create_example(get_current_word())
    word_index += 1



def main():
    """The fucntion that is called in a loop making this an a continous app"""
    global current_phrase
    global current_screen
    global val                  # all global stuff I use here
    global result_text
    global input_text
    running = True
    
    # keeps the game in a loop
    while running:
        screen.fill(WHITE)
        
        '''The next section of code we are checking the screen and changing if need be as well as checking for colides points(button presses)
        if specfic collipde events are done then we call the assocated function or change the glocal vars such as current screen
        so on the next loop so we can move to other screen'''
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if current_screen == "main":
                    if button1_rect.collidepoint(event.pos):
                        current_screen = "review"
                    elif button2_rect.collidepoint(event.pos): # the "Do not press button"  not reason just 2 button didn't look good
                        pass  
                    elif button3_rect.collidepoint(event.pos):
                        gen_phrase()
                        current_screen = "phrases"
                if current_screen == "review":
                    active = False
                    if function_button_rect.collidepoint(event.pos):
                        function_action()
                    elif p_button_word_react.collidepoint(event.pos):
                        mp3_play(get_current_word())
                    elif p_button_sent_react.collidepoint(event.pos):
                        create_pronunciation(function_output.split(".")[0])
                        mp3_play(function_output.split(".")[0])
                    elif print_button_rect.collidepoint(event.pos):
                        val = 1
                    elif next_w_button_rect.collidepoint(event.pos):
                        val = 0
                        next_word()
                    elif back_button_r.collidepoint(event.pos):
                        current_screen = "main"
                if current_screen == "phrases":
                    if input_box.collidepoint(event.pos):
                        active = True
                    else:
                        active = False

                    if button_checker.collidepoint(event.pos): 
                        result_text = test_translation(current_phrase, (input_text.lower().strip()))
                        # if input_text.lower().strip() == "please":
                        #     result_text = "You Pass-- will change to global var from func call"
                        # else:
                        #     result_text = "i guess call again but will leave to test"
                    if generation_button_next.collidepoint(event.pos): 
                        generate_new_phrase()
                    if back_button_p.collidepoint(event.pos):
                        current_screen = "main"

            # the write text was a bit more difficult for me so i needed to do it outside of the range of the others here is for the writng in the box and then 
            # sending that to hte LLM so we can compare what you wrote to what a proper translation is
            elif event.type == pygame.KEYDOWN:
                if active:
                    if event.key == pygame.K_RETURN:
                        result_text = test_translation(current_phrase, (input_text.lower().strip()))
                        #else:
                        #    result_text = "Not the magic word"
                        input_text = ""  # Clear input
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]  # RM last character
                    else:
                        input_text += event.unicode  # adds new character
                                
            
        # changes the screen if current screen was ever changed above 
        if current_screen == "main":
            draw_main_screen()
        elif current_screen == "review":
            draw_review_screen()
        elif current_screen == "phrases":
            draw_phrases_screen()
        
        pygame.display.flip() # is basicly .show need to display the pygame
    
    pygame.quit()

if __name__ == "__main__": # used to run
    main()


# Andddd DONEeeee

Playing: le
Playing: Le chat
Playing:  son
Playing: Son livre est intéressant
